# AI as Judge

[G-Eval](https://deepeval.com/docs/metrics-llm-evals) is a framework that uses LLM as a judge to evaluate LLM outputs. The evaluation can be based on any criteria. G-Eval is implemented by a library called [DeepEval](https://deepeval.com/) which includes a broader set of tests.


In [1]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [2]:
from openai import OpenAI
import os

document_folder = "../../05_src/documents/"
blue_cross_file = "the_blue_cross.txt"
file_path = os.path.join(document_folder, blue_cross_file)

with open(file_path, "r", encoding="utf-8") as f:
    blue_cross_text = f.read()

In [3]:
instructions = "You are an helpful assistant that summarizes works of fiction with a quirky and bubbly approach."
PROMPT = """
    Summarize the following story in at most four paragraphs. Please include all key characters and plot points.
    <story>
    {story}
    </story>
    In addition to the summary, add an introduction paragraph where you greet the reader and a conclusion where you share an opinion about the story.
"""

In [4]:
import os
client = OpenAI(base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1', 
                api_key='any value',
                default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')})
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=instructions,
    input=[
        {"role": "user", 
         "content": PROMPT.format(story=blue_cross_text)}
    ],
    temperature=1.2
)

In [5]:
response.output_text

"**Hello, delightful reader!** Are you ready for a little romp through a captivating mystery sprinkled with wit and charm? Grab your magnifying glass, put on your detective hat, and let’s dive into the delightful escapades of G.K. Chesterton's *The Blue Cross*! \n\nIn this tale, we meet the unassuming yet sharp-witted detective Aristide Valentin, who is hot on the heels of the infamous criminal, Flambeau! Adorned in a rather ordinary outfit that cleverly conceals secrets—a loaded revolver, a police card—Valentin navigates the busy streets of London to apprehend Flambeau during the hectic Eucharistic Congress. Flambeau, known for his remarkable height, cunning disguises, and audacious arrival on the crime scene, is masterfully slippery to track down. As the narrative unfolds, Valentin uses a unique approach, opting to roam the twists and turns of the city rather mindlessly, hoping that the very things that distract may just lead him to Flambeau's hidden den of mischief.\n\nThe tale take

# Answer Relevancy

The answer relevancy metric evaluates how relevant the actual output of the LLM app is compared to the provided input. This metric is self-explaining in the sense that the output includes a reason for the metric score.

The metric is calculated as:

$$
AnswerRelevancy=\frac{NumberRelevantStatements}{TotalStatements}
$$

Reference: [Answer Relevancy](https://deepeval.com/docs/metrics-answer-relevancy). 

In [6]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval.models import GPTModel

model = GPTModel(
    model="gpt-4o-mini",
    temperature=0,
    # api_key='any value',
    default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')},
    base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1',
)

metric = AnswerRelevancyMetric(
    threshold=0.7,
    include_reason=True,
    model=model,
    
)

test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text,
    
)

In [7]:
metric.measure(test_case)

Output()

1.0

In [8]:
from IPython.display import display, Markdown
display(Markdown(f'**Score**: {metric.score}'))
display(Markdown(f'**Reason**: {metric.reason}'))

**Score**: 1.0

**Reason**: The score is 1.00 because the response directly addressed the request for a summary of the story, including key characters and plot points, without any irrelevant statements. The summary was concise and well-structured, effectively capturing the essence of the narrative.

# Other Metrics

Other useful metric functions include:

+ [Faithfulness](https://deepeval.com/docs/metrics-faithfulness): evaluates whether the `actual_output` factually aligns with the contents of  `retrieval_context`. 
+ [Contextual Precision](https://deepeval.com/docs/metrics-contextual-precision): evaluates whether nodes in your `retrieval_context` that are relevant to the given input are ranked higher than irrelevant ones. 
+ [Contextual Recall](https://deepeval.com/docs/metrics-contextual-recall): evaluates the extent of which the retrieval_context aligns with the expected_output. 
+ [Contextual Relevancy](https://deepeval.com/docs/metrics-contextual-relevancy): evaluates the overall relevance of the information presented in your retrieval_context for a given input. 

# G-Eval

[G-Eval](https://deepeval.com/docs/metrics-llm-evals) is a framework that uses LLM-as-a-judge with chain-of-thoughts (CoT) to evaluate LLM outputs based on ANY custom criteria. The G-Eval metric is the most versatile type of metric deepeval offers.

In [9]:
instructions = "You are an helpful assistant that specializes in works of fiction."
PROMPT = """
    Based on the story below, answer the question provided.
    <story>
    {story}
    </story>
    <question>
    Who is the main antagonist in the story and what motivates their actions?
    </question>
"""

In [10]:
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=instructions,
    input=[
        {"role": "user", 
         "content": PROMPT.format(story=blue_cross_text)}
    ],
    temperature=0.7
)

In [11]:
response.output_text

'The main antagonist in "The Blue Cross" is Flambeau, a notorious criminal known for his clever disguises and elaborate thefts. His motivation in the story is primarily driven by greed and the thrill of the chase. Flambeau aims to steal a valuable silver cross adorned with sapphires that a priest, Father Brown, is transporting to a congress in London. \n\nFlambeau\'s character is marked by his physical prowess and cunning, as he often employs elaborate schemes to execute his crimes. His desire to prove his superiority over law enforcement, particularly over the renowned detective Valentin, also adds a layer of motivation to his actions. Ultimately, Flambeau\'s actions reflect the classic traits of a clever antagonist who seeks both financial gain and the excitement of outsmarting his pursuers.'

## Evaluation Criteria

The most straightforward way to establish a metric is by using a single criteria.

In [12]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the context.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

In [13]:
test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)
evaluate(test_cases=[test_case], metrics=[correctness_metric])

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ✅ Correctness [GEval] (score: 0.9244655682049533, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The response accurately identifies Flambeau as the main antagonist and provides a clear explanation of his motivations, including greed and the thrill of the chase. It also highlights his cunning nature and desire to outsmart law enforcement, which aligns well with the context of the story. However, it could have briefly mentioned Flambeau's relationship with Father Brown to enhance the understanding of his motivations further., error: None)

For test case:

  - input: 
    Based on the story below, answer the question provided.
    <story>
    
The Blue Cross
G. K. Chesterton
Between the silver ribbon of morning and the green glittering ribbon of sea, the boat touched
Harwich and let loose a swarm of folk like flies, among whom the man we must follow was by no
means conspicuous—nor wished to be. There was nothing notable about him, except a sli

✓ Evaluation completed 🎉! (time taken: 9.45s | token cost: 0.00167805 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Correctness [GEval]', threshold=0.5, success=True, score=0.9244655682049533, reason="The response accurately identifies Flambeau as the main antagonist and provides a clear explanation of his motivations, including greed and the thrill of the chase. It also highlights his cunning nature and desire to outsmart law enforcement, which aligns well with the context of the story. However, it could have briefly mentioned Flambeau's relationship with Father Brown to enhance the understanding of his motivations further.", strict_mode=False, evaluation_model='gpt-4o-mini', error=None, evaluation_cost=0.00167805, verbose_logs='Criteria:\nDetermine whether the actual output is factually correct based on the context. \n \nEvaluation Steps:\n[\n    "1. Identify the context provided in the input to establish the factual basis for evaluation.",\n    "2. Analyze the actual output to determine if i

## Evaluation Steps 

G-Eval is flexible in many ways: notice that we can establish an evaluation criteria or a set of evaluation steps, that can help in guiding the model to follow specific steps to perform the evaluation.

In [14]:
...

correctness_metric = GEval(
    name="Correctness",
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'input'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are not OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

In [15]:
test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)
result = evaluate(test_cases=[test_case], metrics=[correctness_metric])

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ✅ Correctness [GEval] (score: 0.827713912960153, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The response accurately identifies Flambeau as the main antagonist and provides a clear motivation for his actions, including greed and the thrill of the chase. It also highlights his cunning nature and desire to outsmart law enforcement, which aligns well with the story's themes. However, it could benefit from more detail regarding Flambeau's specific actions and the context of his motivations, as the evaluation steps emphasize the importance of detail and avoiding vagueness., error: None)

For test case:

  - input: 
    Based on the story below, answer the question provided.
    <story>
    
The Blue Cross
G. K. Chesterton
Between the silver ribbon of morning and the green glittering ribbon of sea, the boat touched
Harwich and let loose a swarm of folk like flies, among whom the man we must follow was by no
means conspicuous—nor wished to be. 

✓ Evaluation completed 🎉! (time taken: 12.04s | token cost: 0.0015993000000000001 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [16]:
result.model_dump()

{'test_results': [{'name': 'test_case_0',
   'success': True,
   'metrics_data': [{'name': 'Correctness [GEval]',
     'threshold': 0.5,
     'success': True,
     'score': 0.827713912960153,
     'reason': "The response accurately identifies Flambeau as the main antagonist and provides a clear motivation for his actions, including greed and the thrill of the chase. It also highlights his cunning nature and desire to outsmart law enforcement, which aligns well with the story's themes. However, it could benefit from more detail regarding Flambeau's specific actions and the context of his motivations, as the evaluation steps emphasize the importance of detail and avoiding vagueness.",
     'strict_mode': False,
     'evaluation_model': 'gpt-4o-mini',
     'error': None,
     'evaluation_cost': 0.0015993000000000001,
     'verbose_logs': 'Criteria:\nNone \n \nEvaluation Steps:\n[\n    "Check whether the facts in \'actual output\' contradicts any facts in \'input\'",\n    "You should also 